In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44146
2,Huelva,Confirmados PDIA 14 días,1600
3,Huelva,Tasa PDIA 14 días,"311,78751680729584"
4,Huelva,Confirmados PDIA 7 días,514
5,Huelva,Tasa PDIA 7 dias,"100,16173977434379"
6,Huelva,Total Confirmados,44344
7,Huelva,Curados,40127
8,Huelva,Fallecidos,403


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44146.0


/tmp/ipykernel_3401/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12269.0


/tmp/ipykernel_3401/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3401/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3401/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3401/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1229 personas en los últimos 7 días 

Un positivo PCR cada 364 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44146.0,514.0,1600.0,513170.0,100.161740,311.787517,61.0
Condado-Campiña,13763.0,145.0,411.0,156231.0,92.811286,263.071990,25.0
Huelva-Costa,26064.0,312.0,1032.0,289548.0,107.754155,356.417589,23.0
Sierra de Huelva-Andévalo Central,3908.0,50.0,137.0,67391.0,74.193883,203.291241,13.0
Isla Cristina,3016.0,35.0,133.0,21393.0,163.604917,621.698686,9.0
Palma del Condado (La),1404.0,17.0,52.0,10801.0,157.392834,481.436904,5.0
Huelva (capital),12269.0,117.0,395.0,143837.0,81.342075,274.616406,5.0
Almonte,2222.0,25.0,58.0,24507.0,102.011670,236.667075,4.0
Escacena del Campo,228.0,9.0,20.0,2287.0,393.528640,874.508089,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Linares de la Sierra,11.0,1.0,6.0,262.0,381.679389,2290.076336,0.0
Cañaveral de León,28.0,2.0,9.0,396.0,505.050505,2272.727273,NaN
Almendro (El),57.0,9.0,17.0,840.0,1071.428571,2023.809524,0.0
Rosal de la Frontera,69.0,10.0,23.0,1697.0,589.275192,1355.332940,2.0
Cabezas Rubias,51.0,3.0,7.0,706.0,424.929178,991.501416,0.0
Escacena del Campo,228.0,9.0,20.0,2287.0,393.528640,874.508089,3.0
Isla Cristina,3016.0,35.0,133.0,21393.0,163.604917,621.698686,9.0
Cartaya,1836.0,46.0,121.0,20083.0,229.049445,602.499627,3.0
Ayamonte,1972.0,23.0,122.0,21104.0,108.984079,578.089462,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Rociana del Condado,655.0,1.0,9.0,7939.0,12.596045,113.364404,0.0,0.111111
Bollullos Par del Condado,1256.0,4.0,33.0,14387.0,27.802878,229.373740,1.0,0.121212
Linares de la Sierra,11.0,1.0,6.0,262.0,381.679389,2290.076336,0.0,0.166667
Ayamonte,1972.0,23.0,122.0,21104.0,108.984079,578.089462,3.0,0.188525
Paterna del Campo,193.0,1.0,5.0,3457.0,28.926815,144.634076,0.0,0.200000
Cañaveral de León,28.0,2.0,9.0,396.0,505.050505,2272.727273,0.0,0.222222
Palos de la Frontera,987.0,4.0,17.0,11742.0,34.065747,144.779424,1.0,0.235294
Nerva,247.0,1.0,4.0,5169.0,19.346102,77.384407,1.0,0.250000
Lepe,2932.0,31.0,121.0,27880.0,111.190818,434.002869,1.0,0.256198
